# AAI-540 Group 6 Project<br>
# VenueSignal<br>
<br>

#1) Setup, Configuration, and Connect 

In [4]:
#@title Setup and Import libraries
import pandas as pd
import numpy as np
import os
import json
import re
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# AWS
import boto3
from pyathena import connect
from pyathena.pandas.cursor import PandasCursor

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("Libraries imported successfully")

Libraries imported successfully


In [5]:
#@title S3 Bucket Configuration
# The bucket is located at s3://yelp-aai540-group6/yelp-dataset/
# The ARN for the bucket is arn:aws:s3:::yelp-aai540-group6
S3_BUCKET = 'yelp-aai540-group6'
DATABASE_NAME = 'yelp-dataset'


In [9]:
#@title Athena Configuration
ATHENA_OUTPUT = f's3://{S3_BUCKET}/athena-results/'
REGION = 'us-east-1'  

In [11]:
from IPython.display import display, HTML

display(
    HTML(
        f'<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{S3_BUCKET}/yelp-dataset/json/?region={REGION}&tab=overview">S3 Bucket - Yelp Dataset</a></b>'
    )
)

# Release Resources

In [12]:

%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [1]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}

<IPython.core.display.Javascript object>